<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from utils import lossFunc, relativeErr
from models import SymbolicDiffusion, PointNetConfig
import torch
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
n_embd = 512
timesteps = 1000
batch_size = 1
learning_rate = 1e-4
num_epochs = 10
blockSize = 32
testBlockSize = 400
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
addVars = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
import numpy as np
import glob
from utils import processDataFiles, CharDataset
import random

files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_train.json")
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>',':']) # extract unique characters from the text before converting the text to a list, # T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(trainText, blockSize, chars, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

data has 498795 examples, 49 unique.


In [20]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points, variables = train_dataset.__getitem__(idx)
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

id:58894
outputs:C*exp(C*x1)+C>__________________
variables:1


In [21]:
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_test.json")
textTest = processDataFiles(files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, testBlockSize, chars, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points, variables = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        num_workers=2
    )

data has 967 examples, 49 unique.
tensor(-2.2293) tensor(5.6586)
id:104
outputs:-1.6019981184370435*x1+2.087303785067711>_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
variables:1


In [31]:
import torch
import json
from scipy.optimize import minimize
from math import log

@torch.no_grad()
def test_model(model, test_loader, textTest, train_dataset, device):
    results = {'target': [], 'predicted_skeleton' : [], 'predicted': [], 'error': []}

    for batch_idx, (_, tokens, points, variables) in enumerate(test_loader):
        points = points.to(device)    # [B, 2, 250]
        tokens = tokens.to(device)    # [B, L]
        variables = variables.to(device)  # [B]

        generated_tokens = model.sample(points, variables, device)

        for i in range(batch_size):
            # Ground truth
            eq = ''.join([train_dataset.itos[int(i)] for i in tokens[0]])
            eq = eq.strip(train_dataset.paddingToken).split('>')
            eq = eq[0] #if len(eq[0])>=1 else eq[1]
            eq = eq.strip('<').strip(">")

            # Predicted
            predicted_tokens = generated_tokens[i].cpu().numpy()
            predicted = ''.join([train_dataset.itos[int(idx)] for idx in predicted_tokens])
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0]) >= 1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            predicted = predicted.replace('Ce', 'C*e')
            predicted_skeleton = predicted


            t = json.loads(textTest[i])
            # train a regressor to find the constants (too slow)
            c = [1.0 for i,x in enumerate(predicted) if x=='C'] # initialize coefficients as 1
            # c[-1] = 0 # initialize the constant as zero
            b = [(-2,2) for i,x in enumerate(predicted) if x=='C']  # bounds on variables
            try:
                if len(c) != 0:
                    # This is the bottleneck in our algorithm
                    # for easier comparison, we are using minimize package
                    cHat = minimize(lossFunc, c, #bounds=b,
                                args=(predicted, t['X'], t['Y']))

                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except ValueError:
                raise 'Err: Wrong Equation {}'.format(predicted)
            except Exception as e:
                raise 'Err: Wrong Equation {}, Err: {}'.format(predicted, e)

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
              try:
                  eqTmp = eq + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  YEval = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"TA: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  YEval = 100  # Assign a default value or handle as needed
                  #raise
                  YEval = 100
              Ys.append(YEval)

              try:
                  eqTmp = predicted + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  Yhat = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"PR: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  Yhat = 100  # Assign a default value or handle as needed
              Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats, info=True)


            results['target'].append(eq)
            results['predicted_skeleton'].append(predicted_skeleton)
            results['predicted'].append(predicted)
            results['error'].append(err)

            print(f"\nSample {batch_idx * batch_size + i + 1}:")
            print(f"Target: {eq}")
            print(f"Predicted Skeleton: {predicted_skeleton}")
            print(f"Predicted: {predicted}")
            print(f"Relative Error: {err:.6f}")
            print("-" * 50)

    return results

In [32]:
pconfig = PointNetConfig(
    embeddingSize=n_embd,
    numberofPoints=250,
    numberofVars=1,
    numberofYs=1,
)
model = SymbolicDiffusion(
    pconfig=pconfig,
    vocab_size=train_dataset.vocab_size,
    max_seq_len=32,
    padding_idx=test_dataset.paddingID,
    max_num_vars=9,
    n_layer=4,
    n_head=4,
    n_embd=n_embd,
    timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
).to(device)


print(train_dataset.itos)

model_path = "/content/drive/MyDrive/Colab Notebooks/STAT946_proj/models/best_model.pth"
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print("Testing SymbolicDiffusion model...")
test_results = test_model(model, test_loader, textTest, train_dataset, device)

print("\nSummary:")
for i in range(len(test_results['target'])):
    print(f"Sample {i+1}:")
    print(f"  Target: {test_results['target'][i]}")
    print(f"  Predicted Skeleton: {test_results['predicted_skeleton'][i]}")
    print(f"  Predicted: {test_results['predicted'][i]}")
    print(f"  Error: {test_results['error'][i]:.6f}")

{0: '\n', 1: ' ', 2: '"', 3: '(', 4: ')', 5: '*', 6: '+', 7: ',', 8: '-', 9: '.', 10: '/', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ':', 23: '<', 24: '>', 25: 'C', 26: 'E', 27: 'Q', 28: 'S', 29: 'T', 30: 'X', 31: 'Y', 32: '[', 33: ']', 34: '_', 35: 'c', 36: 'e', 37: 'g', 38: 'i', 39: 'k', 40: 'l', 41: 'n', 42: 'o', 43: 'p', 44: 's', 45: 't', 46: 'x', 47: '{', 48: '}'}
Testing SymbolicDiffusion model...

Sample 1:
Target: -1.4608810703863337*exp(-1.3662906591107051*x1/(1.6570892466721525*x1))+-0.5481326790673482
Predicted Skeleton: **
Predicted: **
Relative Error: 100.000000
--------------------------------------------------
Exception has been occured! EQ: *1.01.0**e+)1.0, OR: *1.01.0**e+)1.0
Exception has been occured! EQ: *1.01.0**e+)1.0, OR: *1.01.0**e+)1.0
Exception has been occured! EQ: *1.01.0**e+)1.0, OR: *1.01.0**e+)1.0
Exception has been occured! EQ: *1.01.0**e+)1.0, OR: *1.01.0**e+)1.0
Exception has been occured! EQ

KeyboardInterrupt: 